# RE RE RE RE FEATURE ENGINEERING OF DATA - adding merchant details

In [2]:
import numpy as np
import pandas as panda
from dateutil import relativedelta
import datetime

In [16]:
train_data_path = 'all_new/train_2.csv'
test_data_path = 'all_new/test_2.csv'
new_transaction_path = 'C:/Users/somak/Documents/somak_python/real_world_use_cases_1/supervised/regression/kaggle/elo/all/new_merchant_transactions.csv'
historical_transaction_path = 'all_new/historical_transactions.csv'
new_treated_transaction_path = 'all_new/treated_new_transaction.csv'
treated_historical_transaction_path = 'all_new/treated_historical_transactions.csv'
merchants_data_path = 'all_new/agg_merchants_1.csv'



In [17]:
new_transaction_data = panda.read_csv(new_transaction_path)


In [18]:
new_transaction_data.merchant_id.isnull().sum(),\
new_transaction_data.shape,\
new_transaction_data.card_id.unique().shape,\
new_transaction_data.loc[(new_transaction_data.merchant_id.isnull())].card_id.unique().shape

(26216, (1963031, 14), (290001,), (26196,))

In [19]:
merchant_data = panda.read_csv(merchants_data_path)
# merchant_data.columns = list(map(lambda x: x if x.startswith('merchant_') else 'merchant_'+x, merchant_data.columns.tolist()))
# ['merchant_'+i for i in merchant_data.columns.tolist() if not i.startswith('merchant_')]
merchant_data.head()

,merchant_category_id,state_id,city_id,subsector_id,category_1,merchant_group_id_nunique,merchant_id_nunique,merchant_id_most_frequent_merchant_id,numerical_1_sum,numerical_1_mean,...,most_sales_range_E_sum,most_sales_range_D_sum,most_sales_range_C_sum,most_sales_range_B_sum,most_sales_range_A_sum,most_recent_purchases_range_E_sum,most_recent_purchases_range_D_sum,most_recent_purchases_range_C_sum,most_recent_purchases_range_B_sum,most_recent_purchases_range_A_sum
0,-1,9,231,27,N,1,1,M_ID_a2cfe4149a,-0.047556,-0.047556,...,1,0,0,0,0,1,0,0,0,0
1,2,1,-1,20,N,4,4,M_ID_eeba19250e,0.037820,0.009455,...,1,3,0,0,0,2,2,0,0,0
2,2,1,246,20,N,4,4,M_ID_628321e5da,0.027905,0.006976,...,1,3,0,0,0,2,2,0,0,0
3,2,2,-1,20,N,4,4,M_ID_0284e321ff,0.672374,0.168093,...,3,0,1,0,0,3,1,0,0,0
4,2,2,76,20,N,15,15,M_ID_6ef27dc5bd,-0.148187,-0.009879,...,4,7,4,0,0,4,11,0,0,0


In [20]:
## deres nothing else we can do now...with this data
## lets look at new transactions first , since it is a smaller data sets

new_transaction_data = panda.read_csv(new_transaction_path)
new_transaction_data_types = new_transaction_data.dtypes
new_transaction_column_types = {}
for index, column_name in enumerate(new_transaction_data.columns.tolist()):
    new_transaction_column_types[column_name] = new_transaction_data_types[index]
del new_transaction_data
new_transaction_data = panda.read_csv(new_transaction_path, dtype = new_transaction_column_types)
new_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [21]:
new_transaction_data[new_transaction_data.merchant_category_id==-1].shape

(20, 14)

In [22]:
new_transaction_data.isnull().sum()

authorized_flag              0
card_id                      0
city_id                      0
category_1                   0
installments                 0
category_3               55922
merchant_category_id         0
merchant_id              26216
month_lag                    0
purchase_amount              0
purchase_date                0
category_2              111745
state_id                     0
subsector_id                 0
dtype: int64

In [23]:
new_transaction_data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
authorized_flag,1963031,1,Y,1963031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
card_id,1963031,290001,C_ID_b7ebee6539,109,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city_id,1.96303e+06,NaN,NaN,NaN,134.387,101.515,-1,69,110,212,347
category_1,1963031,2,N,1899935,NaN,NaN,NaN,NaN,NaN,NaN,NaN
installments,1.96303e+06,NaN,NaN,NaN,0.682964,1.58407,-1,0,1,1,999
category_3,1907109,3,A,922244,NaN,NaN,NaN,NaN,NaN,NaN,NaN
merchant_category_id,1.96303e+06,NaN,NaN,NaN,430.97,246.339,-1,278,367,683,891
merchant_id,1936815,226129,M_ID_00a6ca8a8a,23018,NaN,NaN,NaN,NaN,NaN,NaN,NaN
month_lag,1.96303e+06,NaN,NaN,NaN,1.47652,0.499448,1,1,1,2,2
purchase_amount,1.96303e+06,NaN,NaN,NaN,-0.550969,0.694004,-0.746893,-0.716629,-0.674841,-0.581616,263.157


In [24]:
## we can see missing data in city_id,state_id, subsector_id, installments, and merchant_category_id,category_1, category_2
## additionally we know merchant_id is also missing

## are there any rows in which category_1, merchant_category_id, state_id, city_id and subsector_id is missing 
## where merchant id is provided

new_transaction_data.loc[\
                        
                        (~new_transaction_data.merchant_id.isna() | new_transaction_data.merchant_id!=-1) \
                         &
                         (new_transaction_data.category_1==-1)\
                         &
                         (new_transaction_data.subsector_id==-1)\
                         &
                         (new_transaction_data.state_id==-1)\
                         &
                         (new_transaction_data.city_id==-1)\
                         &
                         (new_transaction_data.category_2==-1)
                         
                        
                        ]

## no..so any place where categoy_1,merchat_category_id, state/city/subsector_id is null is because merchant info not present

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id


In [25]:
merchant_data[merchant_data.merchant_category_id==-1]

,merchant_category_id,state_id,city_id,subsector_id,category_1,merchant_group_id_nunique,merchant_id_nunique,merchant_id_most_frequent_merchant_id,numerical_1_sum,numerical_1_mean,...,most_sales_range_E_sum,most_sales_range_D_sum,most_sales_range_C_sum,most_sales_range_B_sum,most_sales_range_A_sum,most_recent_purchases_range_E_sum,most_recent_purchases_range_D_sum,most_recent_purchases_range_C_sum,most_recent_purchases_range_B_sum,most_recent_purchases_range_A_sum
0,-1,9,231,27,N,1,1,M_ID_a2cfe4149a,-0.047556,-0.047556,...,1,0,0,0,0,1,0,0,0,0


In [26]:
temp_new_transaction_data = new_transaction_data[['card_id','category_1','city_id','state_id','subsector_id','merchant_category_id','merchant_id']]



In [27]:
new_transaction_data.drop(columns = ['category_1','city_id','state_id','subsector_id','merchant_category_id','merchant_id'], inplace = True)

In [28]:
temp_new_transaction_data.head(1)

,card_id,category_1,city_id,state_id,subsector_id,merchant_category_id,merchant_id
0,C_ID_415bb3a509,N,107,9,19,307,M_ID_b0c793002c


In [29]:
def prep_for_aggregate(table):
    
    table['authorized_flag'] = table.authorized_flag.map({'Y':1,'N':0})
#     table['category_1'] = table.category_1.map({'Y':1, 'N':0})
    most_appearing = table.category_2.value_counts().idxmax()
    table.category_2.fillna(value= most_appearing, inplace=True)
    table['enc_category_2_1'] = (table.category_2==1) + 0
    table['enc_category_2_2'] = (table.category_2==2) + 0
    table['enc_category_2_3'] = (table.category_2==3) + 0
    table['enc_category_2_4'] = (table.category_2==4) + 0
    table['enc_category_2_5'] = (table.category_2==5) + 0

    table.drop(columns = ['category_2'], inplace=True)
    most_appearing = table.category_3.value_counts().idxmax()
    table.category_3.fillna(value = most_appearing, inplace = True)
    table['enc_category_3_A'] = (table.category_3=='A') +0
    table['enc_category_3_B'] = (table.category_3=='B') +0
    table['enc_category_3_C'] = (table.category_3=='C') +0
    table.drop(columns = ['category_3'], inplace = True)
    table['enc_purchase_date'] = panda.DatetimeIndex(table.purchase_date).astype(np.int64)*1e-9
    table.drop(columns = ['purchase_date'], inplace = True)
#     most_appearing = table.city_id.value_counts().idxmax()
#     table.city_id.replace({-1:most_appearing}, inplace = True)
#     most_appearing = table.state_id.value_counts().idxmax()
#     table.state_id.replace({-1:most_appearing}, inplace = True)
#     most_appearing = table.subsector_id.value_counts().idxmax()
#     table.subsector_id.replace({-1:most_appearing}, inplace = True)
    
    

In [30]:
prep_for_aggregate(new_transaction_data)
new_transaction_data.head()

,authorized_flag,card_id,installments,month_lag,purchase_amount,enc_category_2_1,enc_category_2_2,enc_category_2_3,enc_category_2_4,enc_category_2_5,enc_category_3_A,enc_category_3_B,enc_category_3_C,enc_purchase_date
0,1,C_ID_415bb3a509,1,1,-0.557574,1,0,0,0,0,0,1,0,1.520780e+09
1,1,C_ID_415bb3a509,1,1,-0.569580,1,0,0,0,0,0,1,0,1.521486e+09
2,1,C_ID_415bb3a509,1,2,-0.551037,1,0,0,0,0,0,1,0,1.524752e+09
3,1,C_ID_415bb3a509,1,1,-0.671925,1,0,0,0,0,0,1,0,1.520416e+09
4,1,C_ID_ef55cf8d4b,1,1,-0.659904,1,0,0,0,0,0,1,0,1.521753e+09


In [31]:
## we will take the 
## sum,mean,max of purchase amount, 
## sum,mean,max of month lag by card_id
## mean of authorized_flag,
## num of authorized ==1
## num of authorized ==0
## sum, mean, max of installments
## mean of category_1
## sum of enc_category_2* and enc_category_3*
from collections import Counter

def count_authorized_one(x):
    return x.eq(1).sum()
def count_authorized_zero(x):
    return x.eq(0).sum()
def count_category_1_one(x):
    return x.eq(1).sum()
def count_category_1_zero(x):
    return x.eq(0).sum()

def count_of_unique_cities(x):
    return x.nunique()

def count_of_unique_subsector_id(x):
    return x.nunique()

def count_of_unique_state_id(x):
    return x.nunique()

def most_frequented_merchant(x):
    c = Counter(x)
    return c.most_common()[0][0]

In [32]:

# aggregate = {
    
# 'purchase_amount' : {'new_purchase_amount_sum':'sum','new_purchase_amount_mean': 'mean','new_purchase_amount_max':'max'},
# 'month_lag' : {'new_month_lag_sum':'sum', 'new_month_lag_mean':'mean','new_month_lag_max':'max'} ,
# 'enc_category_2_1':{'new_category_2_1_sum':'sum'},
# 'enc_category_2_2':{'new_category_2_2_sum':'sum'},
# 'enc_category_2_3':{'new_category_2_3_sum':'sum'},
# 'enc_category_2_4':{'new_category_2_4_sum':'sum'},
# 'enc_category_2_5':{'new_category_2_5_sum':'sum'},    
# 'enc_category_3_A':{'new_category_3_A_sum':'sum'},
# 'enc_category_3_B':{'new_category_3_B_sum':'sum'},
# 'enc_category_3_C':{'new_category_3_C_sum':'sum'},
# 'installments': {'new_installments_sum':'sum','new_installments_mean':'mean','new_installments_max':'max'},
# 'authorized_flag': {'new_authorized_flag_mean':'mean', 'new_authorized_flag_count_authorized_one':count_authorized_one, 'new_authorized_flag_count_authorized_zero':count_authorized_zero},
# 'category_1' :{'new_category_1_mean':'mean', 'new_category_1_count_category_1_one':count_category_1_one, 'new_category_1_count_category_1_zero':count_category_1_zero},
# 'enc_purchase_date' : {'new_purchase_date_mean':'mean','new_purchase_date_min':'min','new_purchase_date_max':'max'},
# 'city_id' : {'new_city_id_nunique':count_of_unique_cities},
# 'state_id': {'new_state_id_nunique':count_of_unique_state_id},
# 'subsector_id' : {'new_subsector_id_nunique':count_of_unique_subsector_id}
    
# }



aggregate = {
    
'purchase_amount' : ['sum','mean','max'],
'month_lag' : ['sum','mean','max'] ,
'enc_category_2_1':['sum'],
'enc_category_2_2': ['sum'],
'enc_category_2_3': ['sum'],
'enc_category_2_4': ['sum'],
'enc_category_2_5': ['sum'],    
'enc_category_3_A': ['sum'],
'enc_category_3_B': ['sum'],
'enc_category_3_C': ['sum'],
'installments': ['sum','mean','max'],
'authorized_flag': ['mean', count_authorized_one, count_authorized_zero],
# 'category_1' :['mean', count_category_1_one, count_category_1_zero],
'enc_purchase_date' : ['mean','min','max',np.ptp],
# 'city_id' : [count_of_unique_cities],
# 'state_id': [count_of_unique_state_id],
# 'subsector_id' : [count_of_unique_subsector_id],
# 'merchant_id' : [most_frequented_merchant]
    
}


In [34]:
group_by = new_transaction_data.groupby(['card_id'])

In [35]:
len(new_transaction_data)

1963031

In [36]:
t = group_by.agg(aggregate)

In [37]:
t.head()

purchase_amount                     month_lag                \
                            sum      mean       max       sum      mean max   
card_id                                                                       
C_ID_00007093c1       -1.328524 -0.664262 -0.656749         4  2.000000   2   
C_ID_0001238066      -14.850055 -0.571156 -0.078318        35  1.346154   2   
C_ID_0001506ef0       -1.447354 -0.723677 -0.715352         2  1.000000   1   
C_ID_0001793786       -0.229620 -0.007407  3.129932        41  1.322581   2   
C_ID_000183fdda       -6.590778 -0.599162 -0.107680        14  1.272727   2   

                enc_category_2_1 enc_category_2_2 enc_category_2_3  \
                             sum              sum              sum   
card_id                                                              
C_ID_00007093c1                1                0                1   
C_ID_0001238066               23                0                0   
C_ID_0001506ef0                0                0                2   
C_ID_0001793786               17                8                5   
C_ID_000183fdda                0                0               11   

                enc_category_2_4    ...     installments                \
                             sum    ...              sum      mean max   
card_id                             ...                                  
C_ID_00007093c1                0    ...                2  1.000000   1   
C_ID_0001238066                0    ...               42  1.615385  10   
C_ID_0001506ef0                0    ...                0  0.000000   0   
C_ID_0001793786                0    ...                0  0.000000   0   
C_ID_000183fdda                0    ...               16  1.454545   4   

                authorized_flag                                             \
                           mean count_authorized_one count_authorized_zero   
card_id                                                                      
C_ID_00007093c1               1                    2                     0   
C_ID_0001238066               1                   26                     0   
C_ID_0001506ef0               1                    2                     0   
C_ID_0001793786               1                   31                     0   
C_ID_000183fdda               1                   11                     0   

                enc_purchase_date                                         
                             mean           min           max        ptp  
card_id                                                                   
C_ID_00007093c1      1.523023e+09  1.522754e+09  1.523291e+09   537024.0  
C_ID_0001238066      1.522335e+09  1.519923e+09  1.525118e+09  5195343.0  
C_ID_0001506ef0      1.521474e+09  1.521239e+09  1.521710e+09   471152.0  
C_ID_0001793786      1.512327e+09  1.510761e+09  1.514742e+09  3981096.0  
C_ID_000183fdda      1.521578e+09  1.519994e+09  1.525100e+09  5106807.0  

[5 rows x 24 columns]

In [38]:
t_cols=['card_id']
t_cols.extend(['_'.join(i) for i in t.columns.tolist()])

t_cols

['card_id',
 'purchase_amount_sum',
 'purchase_amount_mean',
 'purchase_amount_max',
 'month_lag_sum',
 'month_lag_mean',
 'month_lag_max',
 'enc_category_2_1_sum',
 'enc_category_2_2_sum',
 'enc_category_2_3_sum',
 'enc_category_2_4_sum',
 'enc_category_2_5_sum',
 'enc_category_3_A_sum',
 'enc_category_3_B_sum',
 'enc_category_3_C_sum',
 'installments_sum',
 'installments_mean',
 'installments_max',
 'authorized_flag_mean',
 'authorized_flag_count_authorized_one',
 'authorized_flag_count_authorized_zero',
 'enc_purchase_date_mean',
 'enc_purchase_date_min',
 'enc_purchase_date_max',
 'enc_purchase_date_ptp']

In [39]:
t.reset_index(inplace=True)
t.columns = ['new_'+ i for i in t_cols]
t.head()

,new_card_id,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_month_lag_sum,new_month_lag_mean,new_month_lag_max,new_enc_category_2_1_sum,new_enc_category_2_2_sum,new_enc_category_2_3_sum,...,new_installments_sum,new_installments_mean,new_installments_max,new_authorized_flag_mean,new_authorized_flag_count_authorized_one,new_authorized_flag_count_authorized_zero,new_enc_purchase_date_mean,new_enc_purchase_date_min,new_enc_purchase_date_max,new_enc_purchase_date_ptp
0,C_ID_00007093c1,-1.328524,-0.664262,-0.656749,4,2.000000,2,1,0,1,...,2,1.000000,1,1,2,0,1.523023e+09,1.522754e+09,1.523291e+09,537024.0
1,C_ID_0001238066,-14.850055,-0.571156,-0.078318,35,1.346154,2,23,0,0,...,42,1.615385,10,1,26,0,1.522335e+09,1.519923e+09,1.525118e+09,5195343.0
2,C_ID_0001506ef0,-1.447354,-0.723677,-0.715352,2,1.000000,1,0,0,2,...,0,0.000000,0,1,2,0,1.521474e+09,1.521239e+09,1.521710e+09,471152.0
3,C_ID_0001793786,-0.229620,-0.007407,3.129932,41,1.322581,2,17,8,5,...,0,0.000000,0,1,31,0,1.512327e+09,1.510761e+09,1.514742e+09,3981096.0
4,C_ID_000183fdda,-6.590778,-0.599162,-0.107680,14,1.272727,2,0,0,11,...,16,1.454545,4,1,11,0,1.521578e+09,1.519994e+09,1.525100e+09,5106807.0


In [40]:
t.rename(columns = {'new_card_id':'card_id'}, inplace=True)
t.head()

,card_id,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_month_lag_sum,new_month_lag_mean,new_month_lag_max,new_enc_category_2_1_sum,new_enc_category_2_2_sum,new_enc_category_2_3_sum,...,new_installments_sum,new_installments_mean,new_installments_max,new_authorized_flag_mean,new_authorized_flag_count_authorized_one,new_authorized_flag_count_authorized_zero,new_enc_purchase_date_mean,new_enc_purchase_date_min,new_enc_purchase_date_max,new_enc_purchase_date_ptp
0,C_ID_00007093c1,-1.328524,-0.664262,-0.656749,4,2.000000,2,1,0,1,...,2,1.000000,1,1,2,0,1.523023e+09,1.522754e+09,1.523291e+09,537024.0
1,C_ID_0001238066,-14.850055,-0.571156,-0.078318,35,1.346154,2,23,0,0,...,42,1.615385,10,1,26,0,1.522335e+09,1.519923e+09,1.525118e+09,5195343.0
2,C_ID_0001506ef0,-1.447354,-0.723677,-0.715352,2,1.000000,1,0,0,2,...,0,0.000000,0,1,2,0,1.521474e+09,1.521239e+09,1.521710e+09,471152.0
3,C_ID_0001793786,-0.229620,-0.007407,3.129932,41,1.322581,2,17,8,5,...,0,0.000000,0,1,31,0,1.512327e+09,1.510761e+09,1.514742e+09,3981096.0
4,C_ID_000183fdda,-6.590778,-0.599162,-0.107680,14,1.272727,2,0,0,11,...,16,1.454545,4,1,11,0,1.521578e+09,1.519994e+09,1.525100e+09,5106807.0


In [44]:
t.shape, new_transaction_data.shape, temp_new_transaction_data.shape

((290001, 25), (1963031, 14), (1963031, 7))

In [46]:
temp = panda.merge(t,temp_new_transaction_data, on='card_id', how='left')
temp.dropna(inplace=True)
temp.shape, temp.card_id.unique().shape

((1936815, 31), (289057,))

In [27]:
new_transaction_data[['merchant_category_id','state_id','city_id','subsector_id','category_1']].isnull().any()

merchant_category_id    False
state_id                False
city_id                 False
subsector_id            False
category_1              False
dtype: bool

In [33]:
nn = new_transaction_data[['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id']]
tt=panda.concat([t,nn], join='inner',axis = 1)
tt.shape

(290001, 37)

In [35]:
tt[['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id']].head()

,merchant_category_id,state_id,city_id,subsector_id,category_1,card_id,card_id
0,307,9,107,19,0,C_ID_00007093c1,C_ID_415bb3a509
1,307,9,140,19,0,C_ID_0001238066,C_ID_415bb3a509
2,507,9,330,14,0,C_ID_0001506ef0,C_ID_415bb3a509
3,661,9,69,8,1,C_ID_0001793786,C_ID_415bb3a509
4,166,9,69,29,1,C_ID_000183fdda,C_ID_ef55cf8d4b


In [57]:

import random

tt.columns = list(map(lambda x : x+'_'+str(random.randint(1,7)) if x=='card_id'else x, tt.columns.tolist()))

In [58]:
[i for i in tt.columns.tolist() if i.startswith('card_id')]

['card_id_7', 'card_id_6']

In [59]:
# tt[['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id']].columns = ['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id','_card_id']
tt.drop(columns = ['card_id_7'], inplace=True)
tt.rename(columns = {'card_id_6':'card_id'}, inplace =True)
tt[['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id']].head()

,merchant_category_id,state_id,city_id,subsector_id,category_1,card_id
0,307,9,107,19,0,C_ID_415bb3a509
1,307,9,140,19,0,C_ID_415bb3a509
2,507,9,330,14,0,C_ID_415bb3a509
3,661,9,69,8,1,C_ID_415bb3a509
4,166,9,69,29,1,C_ID_ef55cf8d4b


In [62]:
tt['category_1'] = tt.category_1.apply(lambda x: 'Y' if x==1 else 'N')
tt[['merchant_category_id','state_id','city_id','subsector_id','category_1','card_id']].head()

,merchant_category_id,state_id,city_id,subsector_id,category_1,card_id
0,307,9,107,19,N,C_ID_415bb3a509
1,307,9,140,19,N,C_ID_415bb3a509
2,507,9,330,14,N,C_ID_415bb3a509
3,661,9,69,8,N,C_ID_415bb3a509
4,166,9,69,29,N,C_ID_ef55cf8d4b


In [63]:
## at this point we have 2 tables

## tt which is the aggregated data on new transaction data, aggregated by card id

## merchants data which is the merchants data aggregated by 'merchant_category_id','state_id','city_id','subsector_id','category_1'

## we are going to join the above two, to include merchant data on new transaction data

tt.shape, merchant_data.shape

((290001, 36), (46421, 66))

In [64]:
panda.concat([tt,merchant_data], join='inner', axis = 1).shape

(46421, 102)

In [68]:
temp_table = panda.merge(tt, merchant_data, how='left', on =['merchant_category_id','state_id','city_id','subsector_id','category_1'])

In [74]:
temp_table.dropna(inplace = True)
temp_table.isnull().any().sum()

0

In [76]:
temp_table.to_csv('all_new/new_trans_merchant_data.csv', index=False)